# Intro to LangMem

This is an early preview of LangMem, a longterm memory service built by LangChain designed help you eaily build personalized user experiences with LLMs. We will walk through the basic functionality as an orientation to the service, including:

1. Creating memory types
2. Posting messages to the service to trigger memory formation
3. Recalling the memories for use in your bot.

My incorporating this in your chat bot, LangMem will asynchronously help it learn their preferences and interests to improve the quality of its responses and recommendations.

Replicated from [this blog post](https://langchain-ai.github.io/long-term-memory/quick_start/#0-environment-setup)

## 0. Environment Setup
We've created a demo `quickstart` instance for you to try out in this notebook.

While LangMem is still in private alpha, you'll need to be given access to a dedicated LangMem instance from a member LangChain team for more personal use beyond this demo. Reach out on Slack or at support@langchain.dev to receive your connection information.

Then you'll want to install the sdk (we will also use openai in our example chat below).

In [1]:

import json
import uuid
from typing import List

from dotenv import load_dotenv
from langmem import AsyncClient, Client
import openai
from pydantic import BaseModel, Field


load_dotenv()


%load_ext autoreload
%autoreload


With the environment configured, you can create your client. We will connect to OpenAI and to Langmem.

In [2]:
oai_client = openai.AsyncClient()
langmem_client = AsyncClient()

## 1. Creating custom memory types

Memory types let you model your application domain so your bot can retain inforation in a format appropriate to your use case.

LangMem supports both `user` level and `thread`-level memories.

LangMem supports 3 primary types of user memories:

1. Structured "user state" memory to infer and manage a pre-specified user profile
2. Structured "user append state" memory to infer information salient to your application context and query semantically
3. Unstructured user semantic memory

The unstructured semantic memory is enabled by default in each deployment. We will see more of that below.

Enabling structured memory functions is as easy as posting a schema to LangMem. LangMem then automatically manages these custom profiles whenever new messages are sent to the service.

### User State
This is a custom user profile. Instantiate by providing a JSON schema or pydantic model.

In [3]:
class Person(BaseModel):
    name: str = Field(default=None, description="The name of the family member.")
    relation: str = Field(
        default=None, description="The relation of the family member to the user."
    )


class UserProfile(BaseModel):
    preferred_name: str = Field(default=None, description="The user's name.")

    summary: str = Field(
        default="",
        description="A quick summary of how the user would describe themselves.",
    )
    interests: List[str] = Field(
        default_factory=list,
        description="Short (two to three word) descriptions of areas of particular interest for the user. This can be a concept, activity, or idea. Favor broad interests over specific ones.",
    )
    relationships: List[Person] = Field(
        default_factory=Person,
        description="A list of friends, family members, colleagues, and other relationships.",
    )
    other_info: List[str] = Field(
        default_factory=list,
        description="",
    )


user_profile_memory = await langmem_client.create_memory_function(
    UserProfile, target_type="user_state"
)


### User Append State
As the name suggests, the `user_append_state` is an append-only state (meaning the profile is never overwritten) that lets you define schema(s) to represent individual memories which you can later query semantically.

In [4]:
class CoreBelief(BaseModel):
    belief: str = Field(
        default="",
        description="The belief the user has about the world, themselves, or anything else.",
    )
    why: str = Field(description="Why the user believes this.")
    context: str = Field(
        description="The raw context from the conversation that leads you to conclude that the user believes this."
    )


belief_function = await langmem_client.create_memory_function(
    CoreBelief, target_type="user_append_state"
)


class FormativeEvent(BaseModel):
    event: str = Field(
        default="",
        description="The event that occurred. Must be important enough to be formative for the user.",
    )
    impact: str = Field(default="", description="How this event influenced the user.")


event_function = await langmem_client.create_memory_function(
    FormativeEvent, target_type="user_append_state"
)

### User Semantic Memory

There is also a triplet-based user_semantic_memory that is enabled by default. You can turn it off if you don't plan to use it.

In [5]:
functions = langmem_client.list_memory_functions(target_type="user")
semantic_memory = None
async for func in functions:
    if func["type"] == "user_semantic_memory":
        semantic_memory = func


# Uncomment to disable the unstructured memory
# await langmem_client.update_memory_function(semantic_memory["id"], status="disabled")

### Thread Summary

LangMem also supports thread-level memories. We will create them below.

In [6]:
class ConversationSummary(BaseModel):
    title: str = Field(description="Distinct for the conversation.")
    summary: str = Field(description="High level summary of the interactions.")
    topic: List[str] = Field(
        description="Tags for topics discussed in this conversation."
    )


thread_summary_function = await langmem_client.create_memory_function(
    ConversationSummary, target_type="thread_summary"
)

## 2. Starting a conversation
Memories are formed whenever your chat bot posts messages to the service.

Whenever a a user ID is provided in the message metadata, LangMem will automatically create a new user entry and start tracking memories for that user.

In [7]:

johnny_user_id = uuid.uuid4()
jimmy_user_id = uuid.uuid4()
jimmy_username = f"jimmy-{uuid.uuid4().hex[:4]}"
johnny_username = f"johnny-{uuid.uuid4().hex[:4]}"

The following is an example conversation between 1 or more users and an AI¶

In [8]:
# Unique for a given converstaion
thread_id = uuid.uuid4()


async def completion(messages: list):
    stripped_messages = [
        {k: v for k, v in m.items() if k != "metadata"} for m in messages
    ]
    return await oai_client.chat.completions.create(
        model="gpt-3.5-turbo", messages=stripped_messages
    )


messages = [
    {"role": "system", "content": "You are a helpful AI assistant"},
    {
        "role": "user",
        # Names are optional but should be consistent with a given user id, if provided
        "name": jimmy_username,
        "content": "Hey johnny have i ever told you about my older bro steve?",
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "no, you didn't, but I think he was friends with my younger sister sueann",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
    {
        "content": "yeah me and him used to play stickball down in the park before school started. I think it was in 1980",
        "role": "user",
        "name": jimmy_username,
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "That was totally 1979! I remember because i was stuck at home all summer.",
        "role": "user",
        "name": "Jeanne",
        # If the user ID isn't provided, we treat this as a guest message and won't assign memories to the user
    },
    {
        "content": "That was so long ago. I have gotten old and gained 200 pounds since then. I can't even remember who was president. @ai, who was the president in 1980?",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
    {
        "content": "The president of the United States in 1980 was Jimmy Carter.",
        "role": "assistant",
    },
    {
        "content": "Wow ya i forgot that. Stickleball really impacted my life. It's how i first met Jeanne! wonder how my life would have turned out if it hadn't happened that way.",
        "role": "user",
        "name": jimmy_username,
        "metadata": {
            "user_id": str(jimmy_user_id),
        },
    },
    {
        "content": "Yeah wow. That was a big year! @ai could you remind me what else was going on that year?",
        "role": "user",
        "name": johnny_username,
        "metadata": {
            "user_id": str(johnny_user_id),
        },
    },
]

result = await completion(messages)

messages.append(result.choices[0].message)
print(result.choices[0].message.content)

In 1980, some key events and cultural moments included the launch of CNN, the eruption of Mount St. Helens, the introduction of the Rubik's Cube, the release of Pac-Man, the beginning of the Iran-Iraq War, and the election of Ronald Reagan as the President of the United States.


Now that we have the messages, we can share them with LangMem.¶

In [9]:
await langmem_client.add_messages(thread_id=thread_id, messages=messages)

In [10]:
# LangMem will automatically process memories after some delay (~60 seconds), but we can eagerly process the memories as well
await langmem_client.trigger_all_for_thread(thread_id=thread_id)
# You could also trigger for a single user if you'd like
# await langmem_client.trigger_all_for_user(user_id=jimmy_user_id)


#### Fetch messages

You can fetch all the messages in a LangMem thread through that thread's GET endpoint. In this way, LangMem can act as a generic chat bot backend.

In [11]:
messages = langmem_client.list_messages(thread_id=thread_id)
async for message in messages:
    print(message)

{'content': 'You are a helpful AI assistant', 'role': 'system', 'name': None, 'metadata': {'user_id': None, 'timestamp': '2024-04-06T19:50:40.004779', 'id': '85231e4b-168f-407c-8a62-970578ec10e1'}}
{'content': 'Hey johnny have i ever told you about my older bro steve?', 'role': 'user', 'name': 'jimmy-f6b1', 'metadata': {'user_id': '62472af0-4be3-4b7f-939e-8f4db53ad9e3', 'timestamp': '2024-04-06T19:50:40.004813', 'id': '0b4d173d-dcc1-4beb-95c7-68a216f7ef58'}}
{'content': "no, you didn't, but I think he was friends with my younger sister sueann", 'role': 'user', 'name': 'johnny-883b', 'metadata': {'user_id': 'a4242035-3b01-4d6a-bce3-01e11ff1111d', 'timestamp': '2024-04-06T19:50:40.004833', 'id': '47f12d2b-24cc-4aee-9aa6-127b1df9a508'}}
{'content': 'yeah me and him used to play stickball down in the park before school started. I think it was in 1980', 'role': 'user', 'name': 'jimmy-f6b1', 'metadata': {'user_id': '62472af0-4be3-4b7f-939e-8f4db53ad9e3', 'timestamp': '2024-04-06T19:50:40.004


## 3. Query Memory

You can also query the user memory, once it's updated. This may take a few moments - please be patient 😊

To query the unstructured semantic memory, you can provide query text and the number of memories to return.

In [12]:
# Wait a few moments for the memories to process. If this is empty, you'll likely have to wait a bit longer
mems = None
while not mems:
    mem_response = await langmem_client.query_user_memory(
        user_id=jimmy_user_id, text="stickleball", k=3
    )
    mems = mem_response["memories"]
mems

[{'id': '3e8a430f-3a11-406a-8a40-e44eb53e0eb1',
  'created_at': '2024-04-06T19:50:44.759753Z',
  'last_accessed': '2024-04-06T19:50:44.759753Z',
  'text': '',
  'content': {'belief': 'I have an older brother named Steve.',
   'why': 'The user mentioned having an older brother named Steve.',
   'context': 'Hey johnny have i ever told you about my older bro steve?'},
  'scores': {'recency': 0.5, 'importance': 0.5, 'relevance': 0.5}}]

In [13]:
# In a similar way, you can include
# different `user_append_state` memory results
# in the ranked response
mems = await langmem_client.query_user_memory(
    user_id=jimmy_user_id,
    text="stickleball",
    k=3,
    memory_function_ids=[belief_function["id"], event_function["id"]],
)
mems

{'user_id': '62472af0-4be3-4b7f-939e-8f4db53ad9e3',
 'memories': [{'id': '3e8a430f-3a11-406a-8a40-e44eb53e0eb1',
   'created_at': '2024-04-06T19:50:44.759753Z',
   'last_accessed': '2024-04-06T19:50:44.759753Z',
   'text': '',
   'content': {'belief': 'I have an older brother named Steve.',
    'why': 'The user mentioned having an older brother named Steve.',
    'context': 'Hey johnny have i ever told you about my older bro steve?'},
   'scores': {'recency': 0.5, 'importance': 0.5, 'relevance': 0.5}}],
 'state': None,
 'thread_summaries': None}

In [14]:
# For user state (profile) memories, you can make a faster and simple get request
user_state = None
while not user_state:
    user_state = await langmem_client.get_user_memory(
        user_id=jimmy_user_id, memory_function_id=user_profile_memory["id"]
    )
user_state

{'summary': None,
 'interests': None,
 'other_info': ['Played stickball in the park before school around 1980',
  'Meeting Jeanne during stickball significantly impacted life'],
 'relationships': [{'name': 'Steve', 'relation': 'older brother'}],
 'preferred_name': 'Johnny'}

In [15]:
await langmem_client.list_thread_memory(thread_id)

[]

## 4. Use in a later conversations

As you can see, we've extracted some useful information from the previous conversation. We imagine you would fetch these facts in later conversations to provide your bot with additional helpful context about the user.

In [16]:
async def completion_with_memory(messages: list, user_id: uuid.UUID):
    memories = await langmem_client.query_user_memory(
        user_id=user_id, text=messages[-1]["content"], k=3
    )
    facts = "\n".join([mem["text"] for mem in memories["memories"]])
    system_prompt = {
        "role": "system",
        "content": f"You are a helpful assistant. You know the following facts about the user with which you are conversing.\n\n{facts}",
    }
    return await completion([system_prompt] + messages)


messages = [
    {
        "role": "user",
        "name": jimmy_username,
        "content": "Hi there! I'm curious what you remember. What's my brother's name?",
        "metadata": {"user_id": jimmy_user_id},
    }
]
res = await completion_with_memory(messages, user_id=jimmy_user_id)
print(res.choices[0].message.content)

I'm sorry, but I don't have access to personal information about specific individuals. How can I assist you today?


## Conclusion

In this walkthrough, you saved memories for two users to track their interests and other attributes. You did so simply by sending your chat messages to the LangMem service. You then automatically triggered updates to store long-term memories of three forms:

User state "profiles", which follow your custom schema
User append state, which store atomic memories following your custom schema and can be queried semantically.
General-purpose knowledge as semantic triplets
You also tracked thread-scoped summary memories to help you organize your conversational threads.

Finally, let's clean up our work! This demo is a shared workspace :)

In [17]:
## Cleanup

functions = langmem_client.list_memory_functions()



async for func in functions:
    if func["type"] == "user_semantic_memory":
        continue
    # NOTE: this doesn't work because there's no delete_memory_function in async client
    await langmem_client.delete_memory_function(func["id"])

AttributeError: 'AsyncClient' object has no attribute 'delete_memory_function'

In [18]:
functions = langmem_client.list_memory_functions()

In [21]:
async for func in functions:
    print(func)

{'id': 'f9a97c15-b2c8-43f7-9c61-1db658173f1d', 'type': 'user_semantic_memory', 'custom_instructions': None, 'schema': {'name': 'SemanticUserMemory', 'description': 'Remember key information about the user based on the conversation.', 'parameters': {'title': 'extractTriplet', 'description': 'Extract RDF triples from the conversation.', 'type': 'object', 'properties': {'triples': {'title': 'Triples', 'description': 'The facts to extract as memories for future conversations. Be as detailed as possible, while focusing on the user information. Use names over IDs where possible.', 'type': 'array', 'items': {'$ref': '#/definitions/Triple'}}}, 'required': ['triples'], 'definitions': {'Triple': {'title': 'Triple', 'description': 'RDF triple to extract.', 'type': 'object', 'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'predicate': {'title': 'Predicate', 'type': 'string'}, 'object': {'title': 'Object', 'type': 'string'}, 'importance': {'title': 'Importance', 'description': "Ab